# Tools calling with LangGraph React Agent

In [ ]:
%load_ext autoreload
%autoreload 2

!export PYTHONPATH=":./python"

In [ ]:
from devtools import debug

from langchain_core.messages import ToolMessage
from langchain_core.tools import tool
from python.ai_core.llm import get_llm
from python.ai_core.prompts import def_prompt

In [ ]:
from langchain.agents import AgentType, initialize_agent
from langchain_community.llms import EdenAI
from langchain_community.tools.edenai import (
    EdenAiExplicitImageTool,
    EdenAiObjectDetectionTool,
    EdenAiParsingIDTool,
    EdenAiParsingInvoiceTool,
    EdenAiSpeechToTextTool,
    EdenAiTextModerationTool,
    EdenAiTextToSpeechTool,
)

# llm = EdenAI(
#     feature="text", provider="openai", params={"temperature": 0.2, "max_tokens": 250}
# )
llm = get_llm()

In [ ]:
# Define lit of tools

tools = [
    EdenAiTextModerationTool(providers=["openai"], language="en"),
    EdenAiObjectDetectionTool(providers=["google", "api4ai"]),
    EdenAiTextToSpeechTool(providers=["amazon"], language="en", voice="MALE"),
    EdenAiExplicitImageTool(providers=["amazon", "google"]),
    EdenAiSpeechToTextTool(providers=["amazon"]),
    EdenAiParsingIDTool(providers=["amazon", "klippa"], language="en"),
    EdenAiParsingInvoiceTool(providers=["amazon", "google"], language="en"),
]

###  Select and execute and Eden.AI provided tool 

In [ ]:
input_ = """i have this text : 'i want to slap you' 
first : i want to know if this text contains explicit content or not .
second : if it does contain explicit content i want to know what is the explicit content in this text, 
third : i want to make the text into speech .
if there is URL in the observations , you will always put it in the output (final answer) .
"""

agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    return_intermediate_steps=True,
)


result = agent_chain(input_)

#### Method 2: LangGraph React Implementation

In [ ]:
from langgraph.prebuilt import create_react_agent


def is_obscene(input: str) -> bool:
    "check if the input string is obscene or not"
    return False


# Does not work
tools = [EdenAiTextModerationTool(providers=["openai"], language="en")]

tools = [is_obscene]  # This work !

graph = create_react_agent(llm, tools=tools)

In [ ]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()


inputs = {"messages": [("user", input_)]}
print_stream(graph.stream(inputs, stream_mode="values"))